In [8]:
%%capture
!pip install ../Course_Tools/introdl/

In [1]:
%%capture
import os
import torch
from transformers import pipeline
from introdl.utils import get_device, wrap_print_text, config_paths_keys
from introdl.nlp import llm_configure, llm_generate, clear_pipeline, print_pipeline_info, display_markdown

# overload print to wrap text
print = wrap_print_text(print)

device = get_device()

paths = config_paths_keys()


# Introduction to NLP Tasks with Transformer Models

In this notebook we'll demonstrate soutions to some common Natural Langauge Processing (NLP) tasks that use transformer models.  We expand on the material in our NLP textbook Chapter 1 - Hello Transformers.  We'll add a little background about the underlying models.  We'll also demonstrate how these same tasks come be done using a large language model with either "zero-shot prompting" or "few-shot prompting".

Using LLMs for various NLP tasks is common when there isn't much labeled data avaialable.  Zero-shot prompting means that no examples are provided to the LLM.  Few-shot prompting means that a small number of examples are provided to the LLM.

Over the next five lesons we'll go into some of these NLP tasks in detail and a learn a bit about the transfomer neural network architecture.

Here is a customer feedback message that we'll use as an example:

In [2]:
# Sample Text
text = """I ordered the Samsung Galaxy S24 Ultra from Tech Haven, expecting next-day delivery, 
but after three days, I hadn’t even received a shipping update. After waiting 45 minutes on hold, 
customer service told me there was a stock issue—yet no one had informed me! 

When the package finally arrived a week late, it contained a Google Pixel 8 Pro instead. 
The support rep was apologetic but said an exchange would take another two weeks.  

I paid $1,200 for the wrong phone, dealt with delays and poor communication, and now have to wait even longer. 
Tech Haven, you need to do better! Sincerely, Jamie.

To add insult to injury, the customer service representative I spoke with seemed indifferent to my frustration. 
I had to explain my situation multiple times before they even acknowledged the mistake. 
The entire experience has been incredibly disappointing and has left me questioning whether I should ever shop with Tech Haven again. 
It's baffling how a company can operate with such a lack of transparency and efficiency. 
I hope this feedback reaches someone who can make a difference, as no customer should have to go through what I did."""

## NLP Task - Text Classification

Text classification is the process of assigning predefined categories to text. It involves analyzing the content of the text and categorizing it based on its subject, sentiment, or other criteria. One common application of text classification is sentiment analysis, which determines the sentiment expressed in a piece of text, such as positive, negative, or neutral. Sentiment analysis is widely used in customer feedback analysis, social media monitoring, and market research to gauge public opinion and customer satisfaction.

### Sentiment Analysis with a Specialized Model

Here we will let the HuggingFace transformers library provide its default model for sentiment analysis and apply it to our customer feedback.  

In [3]:
# Sentiment Analysis
print("\n**Sentiment Analysis**")
sentiment_pipeline = pipeline("sentiment-analysis", device=device)
print_pipeline_info(sentiment_pipeline)
sentiment_result = sentiment_pipeline(text)
print(sentiment_result)


**Sentiment Analysis**
[{'label': 'NEGATIVE', 'score': 0.9990748167037964}]


In this case, a "BERT" model correctly classified the customer feedback as negative. BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based model developed by Google. It is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers. This allows BERT to understand the context of a word based on its surroundings, making it highly effective for various NLP tasks. The particular model used here is a distilled BERT model that has been fine-tuned on a sentiment dataset. A distilled model is a smaller, faster, and more efficient version of a larger model, trained using knowledge distillation, where the smaller model learns to mimic the outputs of the larger one while retaining most of its performance. In Lesson 9, we'll learn more about the family of transformer models called encoders, which include BERT models.

It's never a bad idea to remove models from memory when they aren't being used:

In [4]:
clear_pipeline(sentiment_pipeline)

Pipeline cleared.  Freed 0.00 MB of CUDA memory


### Sentiment Analysis with a Zero-Shot Prompt

A system prompt is used to give instructions to an LLM while a user prompt is the specific input you want the LLM to respond to.  Here we define a system prompt for sentiment analysis

In [6]:
system_prompt = """You are an expert sentiment analysis model. Analyze the sentiment of the following text. 
Give only a one word response: positive, negative, or neutral."""
user_prompt = f"Text: {text}\nSentiment:"

llm_config = llm_configure("mistral-7B")
response_zero_shot = llm_generate(llm_config, user_prompt, system_prompt=system_prompt)
print(response_zero_shot)

Negative


We can also handle batches of inputs:

In [7]:

customer_comments = [
    "The product quality is amazing, exceeded my expectations!",
    "Terrible service, I will never shop here again.",
    "Fast shipping and great customer support. Highly recommend!",
    "The item arrived damaged and the return process was a nightmare.",
    "I'm very satisfied with my purchase. Will buy again.",
    "The website is user-friendly and the prices are unbeatable.",
    "Received the wrong item and customer service was unhelpful.",
    "Fantastic experience from start to finish.",
    "The product is okay, but not worth the price.",
    "Excellent quality and quick delivery. Very happy!"
]

user_prompts = [f"Text: {comment}\nSentiment:" for comment in customer_comments]

responses_zero_shot = llm_generate(llm_config, user_prompts, system_prompt=system_prompt)

for user_prompt, response_zero_shot in zip(user_prompts, responses_zero_shot):
    print(user_prompt)
    print(response_zero_shot)

Text: The product quality is amazing, exceeded my expectations!
Sentiment:
Positive
Text: Terrible service, I will never shop here again.
Sentiment:
Negative
Text: Fast shipping and great customer support. Highly recommend!
Sentiment:
Positive
Text: The item arrived damaged and the return process was a nightmare.
Sentiment:
Negative
Text: I'm very satisfied with my purchase. Will buy again.
Sentiment:
Positive
Text: The website is user-friendly and the prices are unbeatable.
Sentiment:
Positive
Text: Received the wrong item and customer service was unhelpful.
Sentiment:
Negative
Text: Fantastic experience from start to finish.
Sentiment:
Positive
Text: The product is okay, but not worth the price.
Sentiment:
Negative
Text: Excellent quality and quick delivery. Very happy!
Sentiment:
Positive


In [ ]:
# Named Entity Recognition (NER)
print("\n**Named Entity Recognition**")
ner_pipeline = pipeline("ner", aggregation_strategy="simple", device=device)
print_pipeline_info(ner_pipeline)
ner_result = ner_pipeline(text)
print(ner_result)



**Named Entity Recognition**
Model: dbmdz/bert-large-cased-finetuned-conll03-english, Size: 332,538,889
parameters
[{'entity_group': 'MISC', 'score': 0.9910073, 'word': 'Samsung Galaxy S24
Ultra', 'start': 14, 'end': 38}, {'entity_group': 'ORG', 'score': 0.99012053,
'word': 'Tech Haven', 'start': 44, 'end': 54}, {'entity_group': 'MISC', 'score':
0.992503, 'word': 'Google Pixel 8 Pro', 'start': 325, 'end': 343},
{'entity_group': 'ORG', 'score': 0.96618426, 'word': 'Tech Haven', 'start': 551,
'end': 561}, {'entity_group': 'PER', 'score': 0.98039377, 'word': 'Jamie',
'start': 597, 'end': 602}]


In [14]:
clear_pipeline(ner_pipeline)

Pipeline cleared.  Freed 1330.93 MB of CUDA memory


In [ ]:

# Question Answering
print("\n**Question Answering**")
qa_pipeline = pipeline("question-answering", device=device)
print_pipeline_info(qa_pipeline)
question = "What is the defect?"
qa_result = qa_pipeline(question=question, context=text)
print(qa_result)



**Question Answering**
Model: distilbert/distilbert-base-cased-distilled-squad, Size: 65,192,450
parameters
{'score': 0.11689740419387817, 'start': 222, 'end': 233, 'answer': 'stock
issue'}


In [16]:
clear_pipeline(qa_pipeline)

Pipeline cleared.  Freed 260.77 MB of CUDA memory


In [ ]:

# Translation (English to Spanish)
print("\n**Translation**")
translation_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es", device=device)
print_pipeline_info(translation_pipeline)
translation_result = translation_pipeline(text, max_length=200)
print(translation_result)



**Translation**
Model: Helsinki-NLP/opus-mt-en-es, Size: 77,943,296 parameters
[{'translation_text': 'Pedí el Samsung Galaxy S24 Ultra de Tech Haven, esperando
la entrega del día siguiente, pero después de tres días, ni siquiera había
recibido una actualización de envío. Después de esperar 45 minutos en espera, el
servicio al cliente me dijo que había un problema de existencias — sin embargo,
nadie me había informado! Cuando el paquete finalmente llegó una semana tarde,
que contenía un Google Pixel 8 Pro en su lugar. El representante de apoyo era
apologético, pero dijo que un intercambio tomaría otras dos semanas. Pagué
$1,200 por el teléfono equivocado, trató con retrasos y mala comunicación, y
ahora tienen que esperar aún más. Tech Haven, usted necesita hacer mejor!
Sinceramente, Jamie.'}]


In [18]:
clear_pipeline(translation_pipeline)

Pipeline cleared.  Freed 312.03 MB of CUDA memory


In [ ]:

print("\n**Text Generation**")
generator_pipeline = pipeline("text-generation", device=device)
print_pipeline_info(generator_pipeline)
response = "Dear Jamie, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator_pipeline(prompt, max_length=200)
generated_text = outputs[0]['generated_text']
print(generated_text)



**Text Generation**
Model: openai-community/gpt2, Size: 124,439,808 parameters
I ordered the Samsung Galaxy S24 Ultra from Tech Haven, expecting next-day
delivery,
but after three days, I hadn’t even received a shipping update. After waiting 45
minutes on hold,
customer service told me there was a stock issue—yet no one had informed me!

When the package finally arrived a week late, it contained a Google Pixel 8 Pro
instead.
The support rep was apologetic but said an exchange would take another two
weeks.

I paid $1,200 for the wrong phone, dealt with delays and poor communication, and
now have to wait even longer.
Tech Haven, you need to do better! Sincerely, Jamie.

Customer service response:
Dear Jamie, I am sorry to hear that your order was mixed up.

However, due to the delay, the issue was resolved within 2 weeks, and the order
has been delivered to its original address.

Customer service response:
Dear Jamie, I am sorry to hear that your order was mixed up.

However, due to the

In [20]:
clear_pipeline(generator_pipeline)

Pipeline cleared.  Freed 511.15 MB of CUDA memory


In [ ]:
# Summarization
print("\n**Summarization**")
summarization_pipeline = pipeline("summarization", device=device)
print_pipeline_info(summarization_pipeline)
summarization_result = summarization_pipeline(text, max_length=50, min_length=25, do_sample=False)
print(summarization_result)



**Summarization**
Model: sshleifer/distilbart-cnn-12-6, Size: 305,510,400 parameters
[{'summary_text': ' Tech Haven sent a Samsung Galaxy S24 Ultra to Tech Haven,
expecting next-day delivery . The package arrived a week late, it contained a
Google Pixel 8 Pro instead . An exchange would take another two weeks .'}]


In [23]:
clear_pipeline(summarization_pipeline)

Pipeline cleared.  Freed 1222.24 MB of CUDA memory


In [ ]:
from introdl.nlp import llm_configure, llm_generate
from introdl.utils import wrap_print_text, get_device

print = wrap_print_text(print)

device = get_device()

mistral_config = llm_configure("mistral")
response = llm_generate(mistral_config, "What is the capital of France?")
print(response)



🛑 Unloading model: openai-community/gpt2 from GPU...
✅ Model openai-community/gpt2 has been fully unloaded.
🚀 Loading model: unsloth/mistral-7b-instruct-v0.3-bnb-4bit (this may take a while)...
🟢 Model unsloth/mistral-7b-instruct-v0.3-bnb-4bit loaded successfully.

The capital of France is Paris.


In [ ]:
# Sample Text
text = """I ordered the Samsung Galaxy S24 Ultra from Tech Haven, expecting next-day delivery, 
but after three days, I hadn’t even received a shipping update. After waiting 45 minutes on hold, 
customer service told me there was a stock issue—yet no one had informed me! 

When the package finally arrived a week late, it contained a Google Pixel 8 Pro instead. 
The support rep was apologetic but said an exchange would take another two weeks.  

I paid $1,200 for the wrong phone, dealt with delays and poor communication, and now have to wait even longer. 
Tech Haven, you need to do better! Sincerely, Jamie."""


system_prompt_sentiment = """You are an expert sentiment analysis model. Analyze the sentiment of the following text. 
Give only a one word response: positive, negative, or neutral."""
user_prompt = f"Text: {text}\nSentiment:"

response = llm_generate(mistral_config, user_prompt, system_prompt=system_prompt_sentiment)
print(response)

Negative


In [ ]:
system_prompt_ner = """You are an expert named entity recognition model. Identify and classify the entities in the following text. 
Provide the entities and their types in a JSON format."""
user_prompt_ner = f"Text: {text}\nEntities:"

response_ner = llm_generate(mistral_config, user_prompt_ner, system_prompt=system_prompt_ner)
print(response_ner)

{
  "entities": [
    {
      "name": "Samsung Galaxy S24 Ultra",
      "type": "Product"
    },
    {
      "name": "Tech Haven",
      "type": "Organization"
    },
    {
      "name": "$1,200",
      "type": "CurrencyAmount"
    },
    {
      "name": "Google Pixel 8 Pro",
      "type": "Product"
    }
  ]
}


In [ ]:

question = "What is the issue?"
system_prompt_qa = """You are an expert question answering model. Answer the question based on the context provided. Be succinct"""
user_prompt_qa = f"Context: {text}\nQuestion: {question}\nAnswer:"

response_qa = llm_generate(mistral_config, user_prompt_qa, system_prompt=system_prompt_qa)
print(response_qa)

The issue involves incorrect order delivery (Google Pixel 8 Pro instead of
Samsung Galaxy S24 Ultra), delay in shipment, lack of timely communication about
the stock issue, and further prolonged time for an exchange.


In [ ]:
system_prompt_translation = """You are an expert translation model. Translate the following text from English to Spanish."""
user_prompt_translation = f"Text: {text}\nTranslation:"

response_translation = llm_generate(mistral_config, user_prompt_translation, system_prompt=system_prompt_translation)
print(response_translation)

Estimado Tech Haven, ordené el Samsung Galaxy S24 Ultra esperando entrega en un
día,
sin embargo, después de tres días, aún no había recibido una actualización de
envío. Después de pasar cuarenta y cinco minutos en llamada por línea
telefónica,
el servicio al cliente me informó que existía un problema de stock — sin
embargo, nadie me lo había comunicado previamente!
Cuando finalmente llegó la paquetería retrasada por una semana, contenía un
Google Pixel 8 Pro en su lugar. El representante de soporte se disculpó pero
dijo que para realizar una intercambio necesitarían otras dos semanas.
Pague $1,200 por el teléfono equivocado, tuve que lidiar con retras


In [ ]:
system_prompt_summarization = """You are an expert summarization model. Summarize the following text in a concise manner."""
user_prompt_summarization = f"Text: {text}\nSummary:"

response_summarization = llm_generate(mistral_config, user_prompt_summarization, system_prompt=system_prompt_summarization)
print(response_summarization)

Customer Jamie experienced delayed delivery of the Samsung Galaxy S24 Ulta from
Tech Haven, receiving a Google Pixel 8 Pro instead due to a stock issue that
wasn't communicated. The customer service representative offered an exchange,
but this process will also take additional time. Overall, Jamie is dissatisfied
with the service provided by Tech Haven, citing issues with communication,
delays, and incorrect product shipment.


In [ ]:
system_prompt_generation = """You are an expert text generation model. Generate a customer service response based on the provided context."""
user_prompt_generation = f"Context: {text}\n\nCustomer service response:\n{response}"

response_generation = llm_generate(mistral_config, user_prompt_generation, system_prompt=system_prompt_generation)
print(response_generation)

Dear Jamie,
Thank you for taking the time to share your experience with us at Tech Haven. We
deeply regret the inconvenience you've encountered during your recent purchase
of the Samsung Galaxy S24 Ultra.
We understand that prompt delivery and clear communication are crucial aspects
in maintaining trust with our customers, and we sincerely apologize for falling
short of these expectations. The mix-up with the order is unacceptable, and we
should have kept you updated about the stock issues earlier.
To address this matter, we will immediately process a full refund for the
incorrectly shipped device as well as cover any additional costs related to
returning the Google Pixel 8 Pro back to us. Furthermore, we will prioritize
sending out the correct product (Samsung Galaxy S24 Ultra) without delay, along
with expedited shipping to ensure timely arrival.
As a token of appreciation for your patience throughout this ordeal, we will


In [3]:
!pip install ../Course_Tools/introdl/

Processing c:\users\bagge\my drive\python_projects\ds776_develop_project\ds776\lessons\course_tools\introdl
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for introdl: filename=introdl-1.0-py3-none-any.whl size=41669 sha256=ef0876a17990433a5c0cf7239eec100df7f67d9a9e3cbccbe83cea9d2b439e2b
  Stored in directory: C:\Users\bagge\AppData\Local\Temp\pip-ephem-wheel-cache-2_41wy4l\wheels\f5\d5\0f\11f1d5af64d00defb23fa33cf51b2946a0899888d73571e687
Successfully built introdl
  Attempting uninstall: introdl
    Found existing installation: introdl 1.0
    Uninstalling introdl-1.0:
      Successfully uninstalled introdl-1.0


In [2]:
gemini_config = llm_configure("gemini-flash-lite")

In [ ]:
response = llm_generate(gemini_config, "Tell me a dad joke.")
print(response)

Why don't scientists trust atoms?
Because they make up everything!


In [4]:
response = llm_generate(gemini_config, "I am testing the audio transcription abilities of Gemini. Please tell me three interesting facts about space.")
print(response)

Here are three interesting facts about space:
1.  **There's a planet made of diamond:** Astronomers discovered a planet called
55 Cancri e that is twice the size of Earth and made almost entirely of diamond.
2.  **Space is completely silent:** Sound waves need a medium to travel, and
space is a vacuum.
3.  **The universe is still expanding:** Scientists have observed that the
universe is not only expanding but that the expansion is accelerating.


In [3]:
gemini_config = llm_configure("gemini-flash-lite")

system_prompt = """You are an AI assistant that refines transcriptions.  
The input is the transcription of an audio file.  
Do not answer questions in the input or add any content beyond the original input.  
Do edit the input transcript for clarity but keep topics in roughly the same order.  
Use bullets if appropriate.
"""
input_prompt = """I am testing the audio transcription abilities of Gemini. 
Please tell me three interesting facts about space.
"""
response = llm_generate(gemini_config, input_prompt, system_prompt=system_prompt)
print(response)

I am testing the audio transcription abilities of Gemini.
Please tell me three interesting facts about space.
